<a href="https://colab.research.google.com/github/sarabdar/pySpark-practice/blob/main/Regression_Model_with_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!apt-get install openjdk-11-jdk -y
!pip install pyspark

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  ca-certificates-java fonts-dejavu-core fonts-dejavu-extra java-common
  libatk-wrapper-java libatk-wrapper-java-jni libpcsclite1 libxt-dev libxtst6
  libxxf86dga1 openjdk-11-jdk-headless openjdk-11-jre openjdk-11-jre-headless
  x11-utils
Suggested packages:
  default-jre pcscd libxt-doc openjdk-11-demo openjdk-11-source visualvm
  libnss-mdns fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  | fonts-wqy-zenhei fonts-indic mesa-utils
The following NEW packages will be installed:
  ca-certificates-java fonts-dejavu-core fonts-dejavu-extra java-common
  libatk-wrapper-java libatk-wrapper-java-jni libpcsclite1 libxt-dev libxtst6
  libxxf86dga1 openjdk-11-jdk openjdk-11-jdk-headless openjdk-11-jre
  openjdk-11-jre-headless x11-utils
0 upgraded, 15 newly installed, 0 to remove and 41 not upgraded.
Need to get 122 MB of archives.


In [12]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("linearRegressionModel").getOrCreate()

sc = spark.sparkContext

In [10]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%202/data/FuelConsumptionCo2.csv"

import urllib.request
urllib.request.urlretrieve(url, "FuelConsumptionCo2.csv")

('FuelConsumptionCo2.csv', <http.client.HTTPMessage at 0x7a1185489d60>)

In [13]:
df = spark.read.csv("/content/FuelConsumptionCo2.csv", header= True, inferSchema = True)

df.show()

+---------+------------+------------+------------+----------+---------+------------+--------+--------------------+-------------------+--------------------+------------------------+------------+
|MODELYEAR|        MAKE|       MODEL|VEHICLECLASS|ENGINESIZE|CYLINDERS|TRANSMISSION|FUELTYPE|FUELCONSUMPTION_CITY|FUELCONSUMPTION_HWY|FUELCONSUMPTION_COMB|FUELCONSUMPTION_COMB_MPG|CO2EMISSIONS|
+---------+------------+------------+------------+----------+---------+------------+--------+--------------------+-------------------+--------------------+------------------------+------------+
|     2014|       ACURA|         ILX|     COMPACT|       2.0|        4|         AS5|       Z|                 9.9|                6.7|                 8.5|                      33|         196|
|     2014|       ACURA|         ILX|     COMPACT|       2.4|        4|          M6|       Z|                11.2|                7.7|                 9.6|                      29|         221|
|     2014|       ACURA|  ILX 

In [14]:
df.printSchema()

root
 |-- MODELYEAR: integer (nullable = true)
 |-- MAKE: string (nullable = true)
 |-- MODEL: string (nullable = true)
 |-- VEHICLECLASS: string (nullable = true)
 |-- ENGINESIZE: double (nullable = true)
 |-- CYLINDERS: integer (nullable = true)
 |-- TRANSMISSION: string (nullable = true)
 |-- FUELTYPE: string (nullable = true)
 |-- FUELCONSUMPTION_CITY: double (nullable = true)
 |-- FUELCONSUMPTION_HWY: double (nullable = true)
 |-- FUELCONSUMPTION_COMB: double (nullable = true)
 |-- FUELCONSUMPTION_COMB_MPG: integer (nullable = true)
 |-- CO2EMISSIONS: integer (nullable = true)



In [15]:
model_year_corr = df.stat.corr('MODELYEAR', 'CO2EMISSIONS')

print(model_year_corr)

nan


Why? 'nan'   


Look all the years are 2014

In [16]:
# confirming

model_year_other_than_2014 = df.filter(df['MODELYEAR'] > 2014)

model_year_other_than_2014.show()

+---------+----+-----+------------+----------+---------+------------+--------+--------------------+-------------------+--------------------+------------------------+------------+
|MODELYEAR|MAKE|MODEL|VEHICLECLASS|ENGINESIZE|CYLINDERS|TRANSMISSION|FUELTYPE|FUELCONSUMPTION_CITY|FUELCONSUMPTION_HWY|FUELCONSUMPTION_COMB|FUELCONSUMPTION_COMB_MPG|CO2EMISSIONS|
+---------+----+-----+------------+----------+---------+------------+--------+--------------------+-------------------+--------------------+------------------------+------------+
+---------+----+-----+------------+----------+---------+------------+--------+--------------------+-------------------+--------------------+------------------------+------------+



In [18]:
engine_size_corr = df.stat.corr('ENGINESIZE', 'CO2EMISSIONS')
print("Correlation for ENGINESIZE",engine_size_corr)


cylinders_corr = df.stat.corr('CYLINDERS', 'CO2EMISSIONS')
print("Correlation for CYLINDERS",cylinders_corr)



fuel_consumption_city_corr = df.stat.corr('FUELCONSUMPTION_CITY', 'CO2EMISSIONS')
print("Correlation for FUELCONSUMPTION_CITY",fuel_consumption_city_corr)



fuel_consumption_comb_corr = df.stat.corr('FUELCONSUMPTION_COMB', 'CO2EMISSIONS')
print("Correlation for FUELCONSUMPTION_COMB",fuel_consumption_comb_corr)



fuel_consumption_hwy_corr = df.stat.corr('FUELCONSUMPTION_HWY', 'CO2EMISSIONS')
print("Correlation for FUELCONSUMPTION_HWY",fuel_consumption_hwy_corr)


fuel_consumption_comb_mpg_corr = df.stat.corr('FUELCONSUMPTION_COMB_MPG', 'CO2EMISSIONS')
print("Correlation with FUELCONSUMPTION_COMB_MPG: ",fuel_consumption_comb_mpg_corr)


Correlation for ENGINESIZE 0.8741543683958124
Correlation for CYLINDERS 0.8496845920141154
Correlation for FUELCONSUMPTION_CITY 0.8980385119353927
Correlation for FUELCONSUMPTION_COMB 0.8921285933157569
Correlation for FUELCONSUMPTION_HWY 0.8617479448970436
Correlation with FUELCONSUMPTION_COMB_MPG:  -0.9063942295226142


In [21]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

categorical_cols = ["MAKE", "MODEL", "VEHICLECLASS", "TRANSMISSION", "FUELTYPE"]
index_cols = [c + "_index" for c in categorical_cols]   # will store numeric indices
vec_cols = [c + "_vec" for c in categorical_cols]       # will store one-hot vectors

# StringIndex each categorical column
for c, idx_c in zip(categorical_cols, index_cols):
    df = StringIndexer(inputCol=c, outputCol=idx_c).fit(df).transform(df)

In [24]:
encoder = OneHotEncoder(inputCols=index_cols, outputCols=vec_cols)
df_encoded = encoder.fit(df).transform(df)
df_encoded.show(5, truncate=False)

+---------+-----+----------+------------+----------+---------+------------+--------+--------------------+-------------------+--------------------+------------------------+------------+----------+-----------+------------------+------------------+--------------+---------------+-----------------+----------------+----------------+-------------+
|MODELYEAR|MAKE |MODEL     |VEHICLECLASS|ENGINESIZE|CYLINDERS|TRANSMISSION|FUELTYPE|FUELCONSUMPTION_CITY|FUELCONSUMPTION_HWY|FUELCONSUMPTION_COMB|FUELCONSUMPTION_COMB_MPG|CO2EMISSIONS|MAKE_index|MODEL_index|VEHICLECLASS_index|TRANSMISSION_index|FUELTYPE_index|MAKE_vec       |MODEL_vec        |VEHICLECLASS_vec|TRANSMISSION_vec|FUELTYPE_vec |
+---------+-----+----------+------------+----------+---------+------------+--------+--------------------+-------------------+--------------------+------------------------+------------+----------+-----------+------------------+------------------+--------------+---------------+-----------------+----------------+---

In [25]:
from pyspark.ml.feature import VectorAssembler

numeric_cols = ["MODELYEAR","ENGINESIZE","CYLINDERS","FUELCONSUMPTION_COMB"]
assembler = VectorAssembler(inputCols=numeric_cols + vec_cols, outputCol="features")

final_df = assembler.transform(df_encoded).select("features", "CO2EMISSIONS")
final_df.show(5, truncate=False)

+----------------------------------------------------------------------------+------------+
|features                                                                    |CO2EMISSIONS|
+----------------------------------------------------------------------------+------------+
|(743,[0,1,2,3,31,224,705,732,741],[2014.0,2.0,4.0,8.5,1.0,1.0,1.0,1.0,1.0]) |196         |
|(743,[0,1,2,3,31,224,705,721,741],[2014.0,2.4,4.0,9.6,1.0,1.0,1.0,1.0,1.0]) |221         |
|(743,[0,1,2,3,31,512,705,736,741],[2014.0,1.5,4.0,5.9,1.0,1.0,1.0,1.0,1.0]) |136         |
|(743,[0,1,2,3,31,541,706,720,741],[2014.0,3.5,6.0,11.1,1.0,1.0,1.0,1.0,1.0])|255         |
|(743,[0,1,2,3,31,608,706,720,741],[2014.0,3.5,6.0,10.6,1.0,1.0,1.0,1.0,1.0])|244         |
+----------------------------------------------------------------------------+------------+
only showing top 5 rows



In [26]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol="features", labelCol="CO2EMISSIONS")
lr_model = lr.fit(final_df)

In [28]:
training_summary = lr_model.summary
print("RMSE:", training_summary.rootMeanSquaredError)
print("R2:", training_summary.r2)

RMSE: 4.3263150310182485
R2: 0.9953350768258235
